# Analysis of LSTM forecasting

In [141]:
import matplotlib.pyplot as plt
from matplotlib.dates import HourLocator, DateFormatter
from datetime import timedelta
import os
import seaborn as sns
from matplotlib.ticker import MaxNLocator
from matplotlib.dates import DateFormatter


import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
site_ids_training = [
    "site_0030",
    "site_0027",
    "site_0022",
    "site_0024",
    "site_0009",
    "site_0005",
    "site_0018",
    "site_0006",
    "site_0017",
    "site_0028",
    "site_0012"
]

site_ids_independent = [
    "site_0026", # orientation 1, independent population
    "site_0021", # orientation 4, independent population
]

site_ids_prediction = site_ids_training + site_ids_independent



In [28]:
df_prediction_power = pd.read_csv('../data/shared_data/centralized/prediction/all/prediction_all_power.csv')
df_prediction_acc_energy = pd.read_csv('../data/shared_data/centralized/prediction/all/prediction_all_acc_energy.csv')

df_prediction_power['time'] = pd.to_datetime(df_prediction_power['time'])
df_prediction_power['index'] = df_prediction_power['time']
df_prediction_power.set_index('index', inplace=True)
df_prediction_power = df_prediction_power.sort_values(by='time')

df_prediction_acc_energy['time'] = pd.to_datetime(df_prediction_acc_energy['time'])
df_prediction_acc_energy['index'] = df_prediction_acc_energy['time']
df_prediction_acc_energy.set_index('index', inplace=True)
df_prediction_acc_energy = df_prediction_acc_energy.sort_values(by='time')

df_prediction_power_centralized_all = df_prediction_power
df_prediction_acc_energy_centralized_all = df_prediction_acc_energy

CENTRALIZED_ALL = True
CENTRALIZED_CONTINUAL = False
FEDERATED = False

In [30]:
df_prediction_power = pd.read_csv('../data/shared_data/centralized/prediction/continual/prediction_all_power.csv')
df_prediction_acc_energy = pd.read_csv('../data/shared_data/centralized/prediction/continual/prediction_all_acc_energy.csv')

df_prediction_power['time'] = pd.to_datetime(df_prediction_power['time'])
df_prediction_power['index'] = df_prediction_power['time']
df_prediction_power.set_index('index', inplace=True)
df_prediction_power = df_prediction_power.sort_values(by='time')

df_prediction_acc_energy['time'] = pd.to_datetime(df_prediction_acc_energy['time'])
df_prediction_acc_energy['index'] = df_prediction_acc_energy['time']
df_prediction_acc_energy.set_index('index', inplace=True)
df_prediction_acc_energy = df_prediction_acc_energy.sort_values(by='time')

df_prediction_power_centralized_continual = df_prediction_power
df_prediction_acc_energy_centralized_continual = df_prediction_acc_energy

CENTRALIZED_ALL = False
CENTRALIZED_CONTINUAL = True
FEDERATED = False

In [32]:
df_prediction_power = pd.read_csv('../data/shared_data/federated/prediction/prediction_all_power.csv')
df_prediction_acc_energy = pd.read_csv('../data/shared_data/federated/prediction/prediction_all_acc_energy.csv')

df_prediction_power['time'] = pd.to_datetime(df_prediction_power['time'])
df_prediction_power['index'] = df_prediction_power['time']
df_prediction_power.set_index('index', inplace=True)
df_prediction_power = df_prediction_power.sort_values(by='time')

df_prediction_acc_energy['time'] = pd.to_datetime(df_prediction_acc_energy['time'])
df_prediction_acc_energy['index'] = df_prediction_acc_energy['time']
df_prediction_acc_energy.set_index('index', inplace=True)
df_prediction_acc_energy = df_prediction_acc_energy.sort_values(by='time')

df_prediction_power_federated = df_prediction_power
df_prediction_acc_energy_federated = df_prediction_acc_energy

CENTRALIZED_ALL = False
CENTRALIZED_CONTINUAL = False
FEDERATED = True

In [ ]:
def show_error(data, key, mult, desc):
    data_day = data[(data['time'].dt.time >= pd.to_datetime('06:00').time()) & 
                                    (data['time'].dt.time <= pd.to_datetime('21:00').time())]
    
    error = ((abs((data[key] - data['actual'])) / (data['kwp'] * mult)) * 100)
    
    error_day = (((abs(data_day[key] - data_day['actual'])) / (data_day['kwp'] * mult)) * 100)

    mean_error = error.mean()
    mean_error_day = error_day.mean()

    median_error = error.median()
    median_error_day = error_day.median()

    max_error = error.max()


    print(f'Mean {mean_error_day:.2f}% Median: {median_error_day:.2f}% (06:00-21:00) | Mean: {mean_error:.2f}% Median: {median_error:.2f}% | Max: {max_error:.2f}%    --- Error for {desc} {key}')

for key in ['local']:
    key = f'predicted_{key}'

    show_error(df_prediction_power_centralized_all, key, 1000, "Power (Centralized (All))")
    show_error(df_prediction_acc_energy_centralized_all, key, 1000 * 12, "Acc. Energy (Centralized (All))")

for key in ['local']:
    key = f'predicted_{key}'

    show_error(df_prediction_power_centralized_continual, key, 1000, "Power (Centralized (Continual))")
    show_error(df_prediction_acc_energy_centralized_continual, key, 1000 * 12, "Acc. Energy (Centralized (Continual))")

for key in [ 'global', 'cluster_location', 'cluster_orientation', 'local']:
    key = f'predicted_{key}'

    show_error(df_prediction_power_federated, key, 1000, "Power (Federated)")
    show_error(df_prediction_acc_energy_federated, key, 1000 * 12, "Acc. Energy (Federated)")    

    df_prediction_power_federated_independent = df_prediction_power_federated[df_prediction_power_federated['site_id'].isin(site_ids_independent)]
    df_prediction_acc_energy_federated_independent = df_prediction_acc_energy_federated[df_prediction_acc_energy_federated['site_id'].isin(site_ids_independent)]

    show_error(df_prediction_power_federated_independent, key, 1000, "Power (Federated Independent)")
    show_error(df_prediction_acc_energy_federated_independent, key, 1000 * 12, "Acc. Energy (Federated Independent)")



In [ ]:
# NOTE: all charts are scaled to 1

SAVE_PLOTS = True
SHOW_WEATHER = False

def show_combined_plots(model, plot_info, color, week_data_power, week_data_energy, kwp, max_energy, save_path):
    # Reduced height further
    fig = plt.figure(figsize=(20, 9))
    
    key = f'predicted_{model}'
    round = week_data_power[f'{model}_model_round'].iloc[0]
    
    # Power Plot (top left)
    ax1 = plt.subplot(2, 2, 1)
    ax1.plot(week_data_power['time'], week_data_power['actual'] / (kwp * 1000), 
             label='Actual', linestyle='-', color='black', linewidth=1)
    ax1.plot(week_data_power['time'], week_data_power[key] / (kwp * 1000), 
             label=f'Predicted (Training Round {round})', linestyle='--', color=color, linewidth=1)
    
    if SHOW_WEATHER:
        ax1.plot(week_data_power['time'], week_data_power['solar_rad'] / 1000, label='Solar Rad', linestyle='--', color='grey', linewidth=0.5)
        ax1.plot(week_data_power['time'], week_data_power['clouds'] / 100, label='Clouds', linestyle='--', color='green', linewidth=0.5)
        ax1.plot(week_data_power['time'], week_data_power['snow_depth'] / 1200, label='Snow Depth', linestyle='--', color='red', linewidth=0.5)
        ax1.plot(week_data_power['time'], week_data_power['precip'] / 15, label='Precip', linestyle='--', color='blue', linewidth=0.5)
    
    ax1.set_ylim(0, 1.1)
    ax1.set_title(f'Power Output - {plot_info}')
    ax1.set_xlabel('Time')
    ax1.set_ylabel(f'Power (1 = {kwp}kW)')
    ax1.xaxis.set_major_locator(HourLocator(byhour=[12]))  # Show noon (12:00)
    ax1.xaxis.set_major_formatter(DateFormatter('%y-%m-%d\n12:00'))  # Format as yy-mm-dd 12
    ax1.tick_params(axis='x', rotation=0)
    ax1.legend()
    
    # Energy Plot (top right)
    ax2 = plt.subplot(2, 2, 2)
    ax2.plot(week_data_energy['time'], week_data_energy['actual'] / max_energy, 
             label='Actual', linestyle='-', color='black', linewidth=1)
    ax2.plot(week_data_energy['time'], week_data_energy[key] / max_energy, 
             label=f'Predicted (Training Round {round})', linestyle='--', color=color, linewidth=1)
    
    ax2.set_ylim(0, 1.1)
    ax2.set_title(f'Energy Output - {plot_info}')
    ax2.set_xlabel('Time')
    ax2.set_ylabel(f'Energy (1 = {max_energy/1000:.1f}kWh)')
    ax2.xaxis.set_major_locator(HourLocator(byhour=[12]))  # Show noon (12:00)
    ax2.xaxis.set_major_formatter(DateFormatter('%y-%m-%d\n12:00'))  # Format as yy-mm-dd 12
    ax2.tick_params(axis='x', rotation=0)
    ax2.legend()
    
    # Power Error Plot (bottom left)
    ax3 = plt.subplot(2, 2, 3)
    power_error = (abs((week_data_power[key] - week_data_power['actual'])) / (kwp * 1000)) * 100
    
    # Filter data between 06:00 and 21:00 for power
    week_data_power_day = week_data_power[
        (week_data_power['time'].dt.time >= pd.to_datetime('06:00').time()) & 
        (week_data_power['time'].dt.time <= pd.to_datetime('21:00').time())
    ]
    power_mean_error_day = ((abs(week_data_power_day[key] - week_data_power_day['actual'])) / (kwp * 1000) * 100).mean()
    power_mean_error = power_error.mean()
    
    ax3.plot(week_data_power['time'], power_error, 
             label=f'Power Error (Training Round {round})', linestyle='--', color=color, linewidth=0.5)
    ax3.set_ylim(0, 110)
    ax3.set_title(f'Power Error - {plot_info}')
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Error (%)')
    ax3.xaxis.set_major_locator(HourLocator(byhour=[12]))  # Show noon (12:00)
    ax3.xaxis.set_major_formatter(DateFormatter('%y-%m-%d\n12:00'))  # Format as yy-mm-dd 12
    ax3.tick_params(axis='x', rotation=0)
    ax3.legend()
    ax3.text(0.02, 0.95, f'Mean Error Total: {power_mean_error:.2f}%', 
             transform=ax3.transAxes, fontsize=10, verticalalignment='top', 
             bbox=dict(facecolor='white', alpha=0.8))
    ax3.text(0.02, 0.85, f'Mean Error (06:00-21:00): {power_mean_error_day:.2f}%', 
             transform=ax3.transAxes, fontsize=10, verticalalignment='top', 
             bbox=dict(facecolor='white', alpha=0.8))
    
    # Energy Error Plot (bottom right)
    ax4 = plt.subplot(2, 2, 4)
    energy_error = (abs((week_data_energy[key] - week_data_energy['actual'])) / max_energy) * 100
    
    # Filter data between 06:00 and 21:00 for energy
    week_data_energy_day = week_data_energy[
        (week_data_energy['time'].dt.time >= pd.to_datetime('06:00').time()) & 
        (week_data_energy['time'].dt.time <= pd.to_datetime('21:00').time())
    ]
    energy_mean_error_day = ((abs(week_data_energy_day[key] - week_data_energy_day['actual'])) / max_energy * 100).mean()
    energy_mean_error = energy_error.mean()
    
    ax4.plot(week_data_energy['time'], energy_error, 
             label=f'Energy Error (Training Round {round})', linestyle='--', color=color, linewidth=0.5)
    ax4.set_ylim(0, 110)
    ax4.set_title(f'Energy Error - {plot_info}')
    ax4.set_xlabel('Time')
    ax4.set_ylabel('Error (%)')
    ax4.xaxis.set_major_locator(HourLocator(byhour=[12]))  # Show noon (12:00)
    ax4.xaxis.set_major_formatter(DateFormatter('%y-%m-%d\n12:00'))  # Format as yy-mm-dd 12
    ax4.tick_params(axis='x', rotation=0)
    ax4.legend()
    ax4.text(0.02, 0.95, f'Mean Error Total: {energy_mean_error:.2f}%', 
             transform=ax4.transAxes, fontsize=10, verticalalignment='top', 
             bbox=dict(facecolor='white', alpha=0.8))
    ax4.text(0.02, 0.85, f'Mean Error (06:00-21:00): {energy_mean_error_day:.2f}%', 
             transform=ax4.transAxes, fontsize=10, verticalalignment='top', 
             bbox=dict(facecolor='white', alpha=0.8))
    
    # Adjust spacing between subplots
    plt.tight_layout(h_pad=2, w_pad=3)
    
    if SAVE_PLOTS:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

df_prediction_power['week'] = df_prediction_power['time'].dt.to_period('W')
df_prediction_acc_energy['week'] = df_prediction_acc_energy['time'].dt.to_period('W')

df_prediction_power = df_prediction_power.sort_values(by='time')
df_prediction_acc_energy = df_prediction_acc_energy.sort_values(by='time')

unique_weeks = df_prediction_power['week'].unique()

grouped_data_power = df_prediction_power.groupby(['week', 'site_id'])
grouped_data_acc_energy = df_prediction_acc_energy.groupby(['week', 'site_id'])

for (week, site_id), _ in grouped_data_power:
    kwp = df_prediction_power.loc[df_prediction_power['site_id'] == site_id, 'kwp'].iloc[0]
    max_actual_energy = kwp * 12 * 1000
    
    week_data_power = grouped_data_power.get_group((week, site_id))
    week_data_acc_energy = grouped_data_acc_energy.get_group((week, site_id))
    
    independent_population = not (site_id in site_ids_training)
    independent_population_info = ' [INDEPENDENT POPULATION]' if independent_population else ''
    
    if CENTRALIZED_ALL:
        base_path = '../data/plots/centralized/all'
    elif CENTRALIZED_CONTINUAL:
        base_path = '../data/plots/centralized/continual'
    elif FEDERATED:
        base_path = '../data/plots/federated'

    # ensure path exists
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    plot_info = f'Site {site_id}{independent_population_info}'
    week_path = str(week).replace('/', '_')
    base_path = f'{base_path}/{site_id}_week_{week_path}'
    
    if FEDERATED:
        show_combined_plots('local', f'{plot_info} - Local', 'red', 
                          week_data_power, week_data_acc_energy, kwp, max_actual_energy,
                          f'{base_path}_local_combined.png')
        
        show_combined_plots('cluster_location', f'{plot_info} - Cluster Location', 'green',
                          week_data_power, week_data_acc_energy, kwp, max_actual_energy,
                          f'{base_path}_cluster_location_combined.png')
        
        show_combined_plots('cluster_orientation', f'{plot_info} - Cluster Orientation', 'purple',
                          week_data_power, week_data_acc_energy, kwp, max_actual_energy,
                          f'{base_path}_cluster_orientation_combined.png')
        
        show_combined_plots('global', f'{plot_info} - Global', 'blue',
                          week_data_power, week_data_acc_energy, kwp, max_actual_energy,
                          f'{base_path}_global_combined.png')
        
    elif CENTRALIZED_ALL:
        show_combined_plots('local', f'{plot_info} - Centralized (All)', 'turquoise',
                          week_data_power, week_data_acc_energy, kwp, max_actual_energy,
                          f'{base_path}_local_combined.png')
        
    elif CENTRALIZED_CONTINUAL:
        show_combined_plots('local', f'{plot_info} - Centralized (Continual)', 'aquamarine',
                          week_data_power, week_data_acc_energy, kwp, max_actual_energy,
                          f'{base_path}_local_combined.png')

In [190]:
df_prediction_acc_energy = pd.read_csv('../data/shared_data/federated/training/training_all_acc_energy.csv')

df_prediction_acc_energy['time'] = pd.to_datetime(df_prediction_acc_energy['time'])
df_prediction_acc_energy['index'] = df_prediction_acc_energy['time']
df_prediction_acc_energy['date'] = df_prediction_acc_energy['time'].dt.date
df_prediction_acc_energy.set_index('index', inplace=True)
df_prediction_acc_energy = df_prediction_acc_energy.sort_values(by='time')

df_federated = df_prediction_acc_energy


In [191]:
df_prediction_acc_energy = pd.read_csv('../data/shared_data/centralized/training/continual/training_all_acc_energy.csv')

df_prediction_acc_energy['time'] = pd.to_datetime(df_prediction_acc_energy['time'])
df_prediction_acc_energy['index'] = df_prediction_acc_energy['time']
df_prediction_acc_energy['date'] = df_prediction_acc_energy['time'].dt.date
df_prediction_acc_energy.set_index('index', inplace=True)
df_prediction_acc_energy = df_prediction_acc_energy.sort_values(by='time')

df_centralized_continual = df_prediction_acc_energy

In [175]:
df_prediction_acc_energy = pd.read_csv('../data/shared_data/centralized/training/all/training_all_acc_energy.csv')

df_prediction_acc_energy['time'] = pd.to_datetime(df_prediction_acc_energy['time'])
df_prediction_acc_energy['index'] = df_prediction_acc_energy['time']
df_prediction_acc_energy['date'] = df_prediction_acc_energy['time'].dt.date
df_prediction_acc_energy.set_index('index', inplace=True)
df_prediction_acc_energy = df_prediction_acc_energy.sort_values(by='time')

df_centralized_all = df_prediction_acc_energy

In [192]:
def prepare_production_data(df, prediction_cols):
    df.info()
    df = df.groupby(['site_id', 'date']).last().reset_index()
    #df.info()


    def calculate_max_power(row):
        return 12 * 1000 * row['kwp']  # 12h * 1000W/kW * system size in kWp

    # Add max power column
    df['max_energy'] = df.apply(calculate_max_power, axis=1)
    
    error_cols = []
    training_run_col = []
    predicted_cols = []


    for col in prediction_cols:
        predicted_col = f'predicted_{col}'
        error_col = f'{col}_error'
        error_cols.append(error_col)
        training_run_col.append(f'{col}_model_round')
        predicted_cols.append(predicted_col)
        df[error_col] = abs(df[predicted_col] - df['actual']) / df['max_energy'] * 100

    columns_to_keep = ['site_id', 'date'] + error_cols + training_run_col + ['max_energy']

    df = df[columns_to_keep]

    return df

In [ ]:
df_federated_filtered = prepare_production_data(df_federated, ['local', 'cluster_location', 'cluster_orientation', 'global'])
df_centralized_continual_filtered = prepare_production_data(df_centralized_continual, ['local'])
df_centralized_all_filtered = prepare_production_data(df_centralized_all, ['local'])

df_centralized_continual_filtered.rename(columns={'local_error': 'centralized_continual_error', 'local_model_round': 'centralized_continual_model_round'}, inplace=True)
df_centralized_all_filtered.rename(columns={'local_error': 'centralized_all_error', 'local_model_round': 'centralized_all_model_round'}, inplace=True)

df_combined = df_federated_filtered.merge(df_centralized_continual_filtered, on=['site_id', 'date', 'max_energy'])
df_combined = df_combined.merge(df_centralized_all_filtered, on=['site_id', 'date', 'max_energy'])

df_combined.info()

In [199]:
df_combined['best_model_federated'] = df_combined[['local_error', 'cluster_location_error', 'cluster_orientation_error', 'global_error', 'centralized_continual_error']].idxmin(axis=1)

In [ ]:
df_combined.info()

In [ ]:
df_combined

In [ ]:
def plot_best_model_errors(df, site_id):
    site_data = df[df['site_id'] == site_id].copy()
    site_data['date'] = pd.to_datetime(site_data['date'])
    
    color_map = {
        'local': 'red',
        'cluster_location': 'green',
        'cluster_orientation': 'purple',
        'global': 'blue',
        'centralized_continual': 'aquamarine',
        # 'centralized_all': 'turquoise'
    }
    
    error_columns = ['local_error', 'cluster_location_error', 
                    'cluster_orientation_error', 'global_error', 
                    'centralized_continual_error'] # , 'centralized_all_error']
    
    round_columns = ['local_model_round', 'cluster_location_model_round',
                    'cluster_orientation_model_round', 'global_model_round',
                    'centralized_continual_model_round'] #, 'centralized_all_model_round']
    
    bar_colors = []
    min_errors = []
    valid_dates = []
    x_positions = []
    rounds = []
    position = 0
    
    for idx, row in site_data.iterrows():
        errors = [row[col] for col in error_columns]
        valid_errors = [e for e in errors if not pd.isna(e)]
        
        if valid_errors:
            min_error = min(valid_errors)
            min_errors.append(min_error)
            valid_dates.append(row['date'])
            x_positions.append(position)
            
            # Find which model had minimum error and get its round
            model_name = None
            model_round = None
            for error_col, round_col, error in zip(error_columns, round_columns, errors):
                if error == min_error:
                    model_name = error_col.replace('_error', '')
                    model_round = row[round_col]
                    break
            
            rounds.append(model_round)
            bar_colors.append(color_map[model_name])
            position += 1
    
    plt.figure(figsize=(15, 6))
    bars = plt.bar(x_positions, min_errors, color=bar_colors)
    
    # Add round numbers on top of bars with smaller font size and rotated text
    for idx, rect in enumerate(bars):
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2., height,
                f'{rounds[idx]}',
                ha='center', va='bottom', fontsize=6, rotation=45)  # Smaller font size and rotated
    
    # Clean title and labels for better readability
    plot_title = f'Best Model Errors Over Time for Site {site_id}'.title()  # Capitalize for title case
    plt.title(plot_title)
    
    plt.xlabel('Date')
    plt.ylabel('Error (%)')
    plt.ylim(0, 50)

    # Choose every 3rd date for the x-axis ticks
    tick_interval = 3  # Change this number to adjust the interval (e.g., every 3rd date)
    selected_positions = x_positions[::tick_interval]
    selected_dates = [valid_dates[i] for i in range(0, len(valid_dates), tick_interval)]

    # Format the dates for the x-axis
    plt.xticks(selected_positions, [date.strftime('%Y-%m-%d') for date in selected_dates], rotation=45, ha='right')

    # Adjust the layout to ensure labels fit
    plt.tight_layout()

    # Add a legend with a more readable format
    legend_elements = [plt.Rectangle((0,0),1,1, color=color) 
                      for color in color_map.values()]
    plt.legend(legend_elements, [key.replace('_', ' ').title() for key in color_map.keys()], 
              title='Best Performing Model')
    
    return plt


# ensure path exists
path = '../data/plots/best_model_errors'
if not os.path.exists(path):
    os.makedirs(path)

for site_id in site_ids_training:
    plot = plot_best_model_errors(df_combined, site_id)
    plot_path = f'{path}/{site_id}_best_model_errors.png'
    plot.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.show()
